# Custom attibution models
## Preparing
### Installing programs

We will need the following tools for work:
* DBMS ClickHouse
* Python 2.7 + libs (pandas, requests, plot.ly)
* Jupyter Notebook

#### ClickHouse Installation
ClickHouse can be installed on Linux (you could find a guide here https://clickhouse.yandex/).

Windows and Mac users can run ClickHouse on their machines using Docker: the first line sets up clickhouse-server on port 8123, the second one allows to connect to console client.
```
docker run -d --name clickhouse-server --publish=8123:8123 --publish=9000:9000 yandex/clickhouse-server
docker run -it --rm --link clickhouse-server:9000 yandex/clickhouse-client --host clickhouse-server
```

#### Python and Jupyter Notebook Installation

Python can be downloaded on [official site](https://www.python.org/getit/). 
We will use some libraries besides general ones, the easiest way to install them is to use package manager pip ([how to install pip](https://pip.pypa.io/en/stable/installing/))

```
pip install pandas jupyter requests plotly
```

To run Jupyter Notebook you should execute in terminal the following command:

```jupyter notebook```

After that web-interface will be available by link http://localhost:8888.

### Loading data to ClickHouse using Logs API

The easiest way to load your counter's data to ClickHouse is to use [script on GitHub](https://github.com/yndx-metrika/logs_api_integration).

First of all, you need to specify some params in config.

Config includes basic params and you will need to set up it just once. For this example I've used the following config file:


```
{
	"token" : "<insert>",
	"counter_id": "6375151",
	"visits_fields": [
	    "ym:s:counterID",
	    "ym:s:dateTime",
	    "ym:s:date",
	    "ym:s:visitDuration",
	    "ym:s:bounce",
	    "ym:s:pageViews",
	    "ym:s:goalsID",
	    "ym:s:clientID",
	    "ym:s:lastDirectConditionType",
	    "ym:s:purchaseRevenue",
	    "ym:s:purchaseID",
	    "ym:s:lastTrafficSource",
	    "ym:s:lastAdvEngine",
	    "ym:s:lastSearchEngineRoot",
	    "ym:s:visitID",
	    "ym:s:UTMMedium",
	    "ym:s:startURL"
	],
	"log_level": "DEBUG",
	"retries": 1,
	"retries_delay": 60,
	"clickhouse": {
		"host": "http://localhost:8123",
		"user": "",
		"password": "",
		"visits_table": "visits_all",
		"hits_table": "hits_all",
		"database": "default"
	}
}
```

Some parameters are specified in CLI, such as, source (hits or visits) and date range.

There are several possible modes for this script.
First of all, you could specify params `start_date` and `end_date` to get data for particular period:

```python metrica_logs_api.py -source visits -start_date 2015-12-25 -end_date 2015-12-25```

The other option is to use one of the following predefined modes:
* __history__ - loads the whole  history from the date when counter has been created till the day before yesterday
* __regular__ -  recommended mode for regular process, script will load data for day before yesterday
* __regular_early__ - the same as __regular__ but it loads yesterday data (it's recommended to use regular mode, because yesterday visits can be unfinished)

```python metrica_logs_api.py -source visits -mode regular```

### Useful functions

Let's write some functions for integration with ClickHouse. The first function just returns result from database, the second one is actually a wrapper for the first one, that returns data as pandas dataframe.

In [1]:
HOST = 'http://localhost:8123'
import requests
import pandas as pd
import StringIO

def get_clickhouse_data(query, host = HOST, connection_timeout = 1500):
    r = requests.post(host, params = {'query': query}, timeout = connection_timeout)
    if r.status_code == 200:
        return r.text
    else:
        raise ValueError, r.text
        
def get_clickhouse_df(query, host = HOST, connection_timeout = 1500):
    data = get_clickhouse_data(query, host, connection_timeout) 
    df = pd.read_csv(StringIO.StringIO(data), sep = '\t')
    return df

## Warm-up or a few simple SQL-queries
Let's investigate what data we have in table `visits_all` loaded using Logs API.

In [2]:
q = '''DESCRIBE visits_all'''
print get_clickhouse_data(q)

CounterID	UInt32		
DateTime	DateTime		
Date	Date		
Duration	UInt32		
IsBounce	UInt8		
PageViews	Int32		
GoalsID	Array(UInt32)		
ClientID	UInt64		
DirectConditionType	UInt8		
PurchaseRevenue	Array(Float64)		
PurchaseID	Array(String)		
LastSearchEngine	String		
LastAdvEngine	String		
LastTraficSource	String		
VisitID	UInt64		
UTMMedium	String		
StartURL	String		
ImpressionsProductID	Array(String)		



The full list of fields and description of params can be found in Logs API documentation: https://tech.yandex.ru/metrika/doc/api2/logs/intro-docpage/

In [3]:
q = '''
    SELECT * FROM visits_all 
    LIMIT 5
    FORMAT TabSeparatedWithNames
    '''
get_clickhouse_df(q)

CounterID             DateTime        Date  Duration  IsBounce  PageViews  \
0    6375151  2015-12-01 09:31:49  2015-12-01         4         1          1   
1    6375151  2015-12-01 21:33:12  2015-12-01         0         1          1   
2    6375151  2015-12-01 08:54:22  2015-12-01         1         1          1   
3    6375151  2015-12-01 10:56:43  2015-12-01         0         1          1   
4    6375151  2015-12-01 15:25:47  2015-12-01       171         0          6   

  GoalsID  ClientID  DirectConditionType PurchaseRevenue PurchaseID  \
0      []         0                    0              []         []   
1      []         0                    0              []         []   
2      []         0                    0              []         []   
3      []         0                    0              []         []   
4      []         0                    0              []         []   

  LastSearchEngine LastAdvEngine LastTraficSource              VisitID  \
0           yandex        market               ad  5934905382539739018   
1           yandex           NaN          organic  5935082670770524045   
2           yandex        market               ad  5934896179407679609   
3              NaN           NaN           direct  5934926249142175607   
4              NaN           NaN           direct  5934992373385930594   

  UTMMedium                                           StartURL  \
0       cpc  http://220-volt.ru/catalog-156860/?ref=yamar&y...   
1       NaN  http://m.220-volt.ru/catalog/svetodiodnyj-zana...   
2       cpc  http://220-volt.ru/catalog-210053/?ref=yamar&y...   
3       NaN        http://ulyanovsk.220-volt.ru/catalog-59739/   
4       NaN                 http://m.220-volt.ru/catalog/2-55/   

  ImpressionsProductID  
0           ['156860']  
1                   []  
2           ['210053']  
3                   []  
4                   []

In [4]:
q = '''
    SELECT min(Date), max(Date)
    FROM visits_all
    FORMAT TabSeparatedWithNames
'''
get_clickhouse_df(q)

min(Date)   max(Date)
0  2015-12-01  2015-12-31

Let's calculate total number of visits, users, purchases and revenue.

__Disclaimer: revenue has been multiplied by some random number.__

In [5]:
q = '''
    SELECT 
        count() as TotalVisits,
        uniq(ClientID) as TotalUsers,
        sum(length(PurchaseID)) as TotalPurchases,
        sum(arraySum(PurchaseRevenue)) as TotalRevenue
    FROM visits_all
    FORMAT TabSeparatedWithNames
'''
get_clickhouse_df(q)

TotalVisits  TotalUsers  TotalPurchases  TotalRevenue
0      5451184     2976586           70464    8336006970

## Paths to conversion (purchase)

First of all, let's look at visits with purchases

In [7]:
q = '''
    SELECT 
        LastTraficSource,
        ClientID,
        VisitID
    FROM visits_all
    WHERE (ClientID != 0) AND (length(PurchaseID) != 0)
    LIMIT 5
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

LastTraficSource              ClientID              VisitID
0           direct  14466379051005714998  5939061898048872152
1          organic   1450082044461071361  5939344097664677658
2         referral   1449992588546282437  5939169643029919526
3          organic   1446633457145398431  5939217871051015961
4           direct   1448190872275658371  5939249704137096911

To construct paths we need to define for each purchase which traffic sources client had in visits preceeding purchase. To do this we need to join table `visits_all` with itself using `ClientID`.

In [8]:
q = '''
    SELECT 
        ClientID,
        VisitID,
        PurchaseTime,
        DateTime,
        LastTraficSource
    FROM
        (SELECT 
            ClientID,
            DateTime as PurchaseTime, 
            VisitID
        FROM visits_all
        WHERE (ClientID != 0) AND (length(PurchaseID) != 0))
        ALL LEFT JOIN
        (SELECT 
            ClientID,
            LastTraficSource,
            DateTime
        FROM visits_all
        ORDER BY DateTime)
        USING ClientID
    WHERE PurchaseTime >= DateTime
    LIMIT 5
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

ClientID              VisitID         PurchaseTime  \
0   1446494115200148125  5935388931880519536  2015-12-02 18:19:23   
1   1449082733165609622  5935444959877079158  2015-12-02 22:07:21   
2  14466379051005714998  5939061898048872152  2015-12-13 03:24:43   
3  14466379051005714998  5939061898048872152  2015-12-13 03:24:43   
4   1450082044461071361  5939344097664677658  2015-12-13 22:33:00   

              DateTime LastTraficSource  
0  2015-12-02 18:19:23           direct  
1  2015-12-02 22:07:21               ad  
2  2015-12-04 08:29:26           direct  
3  2015-12-13 03:24:43           direct  
4  2015-12-13 22:33:00          organic

Now let's group rows by `VisitID` and using function `groupArray` make arrays from traffic sources.

In [9]:
q = '''
    SELECT 
        VisitID,
        groupArray(LastTraficSource) as Sources
    FROM
        (SELECT 
            ClientID,
            DateTime as PurchaseTime, 
            VisitID
        FROM visits_all
        WHERE (ClientID != 0) AND (length(PurchaseID) != 0))
        ALL LEFT JOIN
        (SELECT 
            ClientID,
            LastTraficSource,
            DateTime
        FROM visits_all
        ORDER BY DateTime)
        USING ClientID
    WHERE PurchaseTime >= DateTime
    GROUP BY VisitID
    LIMIT 5
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

VisitID                                            Sources
0  5942036913160910687                         ['ad','ad','ad','ad','ad']
1  5944887285864662889                             ['ad','internal','ad']
2  5936327790758690505                                             ['ad']
3  5939577186123655034  ['direct','direct','ad','direct','direct','dir...
4  5935674958795425569                                         ['social']

And finally we need to calculate how many times we see each path.

In [10]:
q = '''
    SELECT 
        Sources, 
        sum(NumPurchases) as TotalNumPurchases
    FROM
        (SELECT 
            any(NumPurchases) as NumPurchases,
            VisitID,
            groupArray(LastTraficSource) as Sources
        FROM
            (SELECT 
                length(PurchaseID) as NumPurchases,
                ClientID,
                DateTime as PurchaseTime, 
                VisitID
            FROM visits_all
            WHERE (ClientID != 0) AND (length(PurchaseID) != 0))
            ALL LEFT JOIN
            (SELECT 
                ClientID,
                LastTraficSource,
                DateTime
            FROM visits_all
            ORDER BY DateTime)
            USING ClientID
        WHERE PurchaseTime >= DateTime
        GROUP BY VisitID)
    WHERE length(Sources) >= 2
    GROUP BY Sources
    ORDER BY TotalNumPurchases DESC
    LIMIT 5
    FORMAT TabSeparatedWithNames
'''

%time get_clickhouse_df(q)

CPU times: user 5.26 ms, sys: 1.73 ms, total: 6.99 ms
Wall time: 2.67 s


Sources  TotalNumPurchases
0            ['ad','ad']               2200
1  ['organic','organic']               2159
2       ['organic','ad']               1701
3    ['direct','direct']                989
4       ['ad','ad','ad']                903

We have achieved our first results :) We have constructed the conversion paths of standard Yandex.Metrika traffic sources.

Actually we want to divide traffic in a bit different channels:
* __CPA__ (`UTMMedium` equals `CPA` or `cpa`)
* __CPC__ (`UTMMedium` equals `cpc`)
* __MarketPlace__ (`UTMMedium` equals `marketplace` or it's ad traffic from Yandex.Market)
* __Retargeting__ (using `DirectConditionType` or by `ref=ga_ref` in URL)
* __Media__ - other ad visits
* Let's group sources Direct, Saved pages and internal in one group __Direct__


Let's try do it in ClickHouse.

In [11]:
q = '''
    SELECT 
        LastTraficSource,
        UTMMedium,
        StartURL,
        LastAdvEngine, 
        LastSearchEngine,
        if(
            lower(UTMMedium) IN ('cpa', 'cpc', 'marketplace'), 
            lower(UTMMedium),
            if(
                LastTraficSource = 'ad', 
                if(
                    LastAdvEngine = 'market', 
                    'marketplace',
                    if(
                        (DirectConditionType = 2) OR (extractURLParameter(StartURL, 'ref') = 'ga_ref'),
                        'retargeting',
                        'media_ad')
                ),
                if(
                   LastTraficSource IN ('direct', 'saved', 'undefined', 'internal'),
                   'direct',
                   LastTraficSource
                ) 
            )
        ) as Source,
        if(Source = 'organic', 
            if(LastSearchEngine IN ('yandex', 'google'), concat('organic ', LastSearchEngine), 'organic other'),
            if(Source IN ('media_ad', 'retargeting', 'cpa', 'cpc'),
                if(
                    LastAdvEngine IN ('market', 'google_adwords', 'yandex'), 
                    concat(Source, concat(' ', LastAdvEngine)),
                    concat(Source, ' other')
                ),
                Source
            )) as SourceDetailed
    FROM visits_all
    LIMIT 10
    FORMAT TabSeparatedWithNames
'''
get_clickhouse_df(q)

LastTraficSource UTMMedium  \
0               ad       cpc   
1          organic       NaN   
2               ad       cpc   
3           direct       NaN   
4           direct       NaN   
5          organic       NaN   
6               ad       NaN   
7               ad       NaN   
8          organic       NaN   
9          organic       NaN   

                                            StartURL   LastAdvEngine  \
0  http://220-volt.ru/catalog-156860/?ref=yamar&y...          market   
1  http://m.220-volt.ru/catalog/svetodiodnyj-zana...             NaN   
2  http://220-volt.ru/catalog-210053/?ref=yamar&y...          market   
3        http://ulyanovsk.220-volt.ru/catalog-59739/             NaN   
4                 http://m.220-volt.ru/catalog/2-55/             NaN   
5                   http://omsk.220-volt.ru/catalog/             NaN   
6     http://m.220-volt.ru/catalog-53213/?ref=ga_ret  google_adwords   
7    http://m.220-volt.ru/catalog-220325/?ref=ga_ret  google_adwords   
8  http://kemerovo.220-volt.ru/catalog/svarochie-...             NaN   
9                 http://220-volt.ru/catalog-136605/             NaN   

  LastSearchEngine    Source           SourceDetailed  
0           yandex       cpc               cpc market  
1           yandex   organic           organic yandex  
2           yandex       cpc               cpc market  
3              NaN    direct                   direct  
4              NaN    direct                   direct  
5           yandex   organic           organic yandex  
6              NaN  media_ad  media_ad google_adwords  
7              NaN  media_ad  media_ad google_adwords  
8           yandex   organic           organic yandex  
9           google   organic           organic google

I have added to the script for conversion paths new traffic sources algorithm and got stats by new channels.

In [12]:
q = '''
SELECT 
    SourcesDetailed, 
    sum(VisitPurchases) as Purchases
    FROM
        (SELECT 
            groupArray(Source) as Sources,
            groupArray(SourceDetailed) as SourcesDetailed,
            VisitID,
            any(ClientID) as ClientID,
            any(DateTime) as StartTime,
            any(VisitRevenue) as VisitRevenue,
            any(VisitPurchases) as VisitPurchases
        FROM
            (SELECT
                ClientID,
                DateTime,
                VisitRevenue,
                VisitPurchases,
                Source,
                SourceDetailed,
                VisitID,
                LastSearchEngine,
                LastAdvEngine
            FROM
                (SELECT
                    ClientID,
                    DateTime as PurchaseTime,
                    arraySum(PurchaseRevenue) as VisitRevenue,
                    length(PurchaseID) as VisitPurchases,
                    LastTraficSource,
                    VisitID,
                    LastSearchEngine,
                    LastAdvEngine
                FROM visits_all 
                WHERE ClientID != 0 AND VisitPurchases != 0)
            ALL LEFT JOIN
                (SELECT
                    ClientID,
                    DateTime,
                    if(
                        lower(UTMMedium) IN ('cpa', 'cpc', 'marketplace'), 
                        lower(UTMMedium),
                        if(
                            LastTraficSource = 'ad', 
                            if(
                                LastAdvEngine = 'market', 
                                'marketplace',
                                if(
                                    (DirectConditionType = 2) OR (extractURLParameter(StartURL, 'ref') = 'ga_ref'),
                                    'retargeting',
                                    'media_ad')
                            ),
                            if(
                               LastTraficSource IN ('direct', 'saved', 'undefined', 'internal'),
                               'direct',
                               LastTraficSource
                            ) 
                        )
                    ) as Source,
                    if(Source = 'organic', 
                        if(LastSearchEngine IN ('yandex', 'google'), concat('organic ', LastSearchEngine), 'organic other'),
                        if(Source IN ('media_ad', 'retargeting', 'cpa', 'cpc'),
                            if(
                                LastAdvEngine IN ('market', 'google_adwords', 'yandex'), 
                                concat(Source, concat(' ', LastAdvEngine)),
                                concat(Source, ' other')
                            ),
                            Source
                        )) as SourceDetailed
                 FROM visits_all 
                 ORDER BY DateTime)
            USING ClientID
            WHERE PurchaseTime >= DateTime)
        GROUP BY VisitID)
    GROUP BY SourcesDetailed
    ORDER BY Purchases DESC
    LIMIT 10
    FORMAT TabSeparatedWithNames
'''

%time get_clickhouse_df(q)

CPU times: user 6.75 ms, sys: 1.82 ms, total: 8.57 ms
Wall time: 6.18 s


SourcesDetailed  Purchases
0                   ['organic yandex']       8532
1                           ['direct']       4707
2                   ['organic google']       3989
3                       ['cpc market']       3690
4                        ['cpa other']       2249
5                       ['cpc yandex']       1966
6                        ['cpc other']       1531
7  ['organic yandex','organic yandex']       1321
8                  ['direct','direct']       1164
9                            ['email']       1044

## Attribution models

### Last click model

In [13]:
q = '''
    SELECT 
        count() as Visits,
        sum(arraySum(PurchaseRevenue)) as Revenue,
        sum(length(PurchaseID)) as Purchases,
        if(
            lower(UTMMedium) IN ('cpa', 'cpc', 'marketplace'), 
            lower(UTMMedium),
            if(
                LastTraficSource = 'ad', 
                if(
                    LastAdvEngine = 'market', 
                    'marketplace',
                    if(
                        (DirectConditionType = 2) OR (extractURLParameter(StartURL, 'ref') = 'ga_ref'),
                        'retargeting',
                        'media_ad')
                ),
                if(
                   LastTraficSource IN ('direct', 'saved', 'undefined', 'internal'),
                   'direct',
                   LastTraficSource
                ) 
            )
        ) as Source,
        if(Source = 'organic', 
            if(LastSearchEngine IN ('yandex', 'google'), concat('organic ', LastSearchEngine), 'organic other'),
            if(Source IN ('media_ad', 'retargeting', 'cpa', 'cpc'),
                if(
                    LastAdvEngine IN ('market', 'google_adwords', 'yandex'), 
                    concat(Source, concat(' ', LastAdvEngine)),
                    concat(Source, ' other')
                ),
                Source
            )) as SourceDetailed
    FROM visits_all
    WHERE ClientID != 0
    GROUP BY Source, SourceDetailed
    FORMAT TabSeparatedWithNames
'''

%time last_raw_df = get_clickhouse_df(q).groupby(['Source', 'SourceDetailed']).sum()

CPU times: user 11.4 ms, sys: 3.49 ms, total: 14.9 ms
Wall time: 2.62 s


In [14]:
last_raw_df_norm = last_raw_df.apply(lambda x: 100.*x/last_raw_df.sum(), axis = 1)

### Custom model by behaviour

Let's write a base function to calculate custom attribution model:
* let's take into account only visits without bounce
* traffic sources with achievment "Addint product to cart" will have +50% of weight
* traffic sources with impressions will have +10% of weight

In [15]:
def get_attribution_data_sample(i, N, where_condition = ''):
    q = '''
    SELECT 
        sum(VisitRevenue*SourceCoefNorm) as Revenue,
        sum(VisitPurchases*SourceCoefNorm) as Purchases,
        sum(SourceCoefNorm) as Visits,
        Source,
        SourceDetailed
    FROM
        (SELECT 
            groupArray(Source) as Sources,
            groupArray(SourceDetailed) as SourcesDetailed,
            groupArray(HasCart) as HasCarts,
            groupArray(HasImpression) as HasImpressions,
            VisitID,
            any(ClientID) as ClientID,
            any(DateTime) as StartTime,
            any(VisitRevenue) as VisitRevenue,
            any(VisitPurchases) as VisitPurchases,
            arrayMap(x, y -> 1 + 0.5*x + 0.1*y, HasCarts, HasImpressions) as SourcesCoefs,
            arraySum(SourcesCoefs) as SourcesCoefsSum,
            arrayMap(x -> x/SourcesCoefsSum, SourcesCoefs) as SourcesCoefsNorm
        FROM
            (SELECT
                ClientID,
                DateTime,
                VisitRevenue,
                VisitPurchases,
                SourceTime,
                Source,
                HasCart,
                HasImpression,
                SourceDetailed,
                VisitID,
                LastSearchEngine,
                LastAdvEngine
            FROM
                (SELECT
                    ClientID,
                    DateTime,
                    arraySum(PurchaseRevenue) as VisitRevenue,
                    length(PurchaseID) as VisitPurchases,
                    LastTraficSource,
                    VisitID,
                    LastSearchEngine,
                    LastAdvEngine
                FROM visits_all SAMPLE 1/{N} OFFSET {i}/{N}
                WHERE ClientID != 0)
            ALL LEFT JOIN
                (SELECT
                    ClientID,
                    DateTime as SourceTime,
                    if(
                        lower(UTMMedium) IN ('cpa', 'cpc', 'marketplace'), 
                        lower(UTMMedium),
                        if(
                            LastTraficSource = 'ad', 
                            if(
                                LastAdvEngine = 'market', 
                                'marketplace',
                                if(
                                    (DirectConditionType = 2) OR (extractURLParameter(StartURL, 'ref') = 'ga_ref'),
                                    'retargeting',
                                    'media_ad')
                            ),
                            if(
                               LastTraficSource IN ('direct', 'saved', 'undefined', 'internal'),
                               'direct',
                               LastTraficSource
                            ) 
                        )
                    ) as Source,
                    if(Source = 'organic', 
                        if(LastSearchEngine IN ('yandex', 'google'), concat('organic ', LastSearchEngine), 'organic other'),
                        if(Source IN ('media_ad', 'retargeting', 'cpa', 'cpc'),
                            if(
                                LastAdvEngine IN ('market', 'google_adwords', 'yandex'), 
                                concat(Source, concat(' ', LastAdvEngine)),
                                concat(Source, ' other')
                            ),
                            Source
                        )) as SourceDetailed,
                    has(GoalsID, 552829) as HasCart,
                    length(ImpressionsProductID) > 0 as HasImpression
                 FROM visits_all SAMPLE 1/{N} OFFSET {i}/{N} {where_condition}
                 )
            USING ClientID
            WHERE SourceTime <= DateTime
            ORDER BY SourceTime)
        GROUP BY VisitID)
    ARRAY JOIN
        SourcesCoefsNorm as SourceCoefNorm,
        Sources as Source,
        SourcesDetailed as SourceDetailed
    GROUP BY Source, SourceDetailed
    ORDER BY Visits DESC
    FORMAT TabSeparatedWithNames
    '''.format(
        i = i, N = N,
        where_condition = where_condition
    )
    # print q
    return get_clickhouse_df(q)

def get_attribution_data(where_condition=''):
    tmp_dfs = []
    for i in range(10):
        tmp_dfs.append(get_attribution_data_sample(i, 10, where_condition))
    
    return pd.concat(tmp_dfs).groupby(['Source', 'SourceDetailed']).sum()

In [16]:
%time not_bounce_df = get_attribution_data(where_condition = 'WHERE IsBounce = 0')

CPU times: user 95.5 ms, sys: 13.9 ms, total: 109 ms
Wall time: 40.1 s


In [17]:
not_bounce_df_norm = not_bounce_df.apply(lambda x: 100.*x/not_bounce_df.sum(), axis = 1)

### Custom model with different weights for channels and time decay

In [18]:
traffic_sources_coeffs = {
    'direct': 0.1,
    'referral': 0.2,
    'social': 0.3,
    'cpa': 0.5,
    'cpc': 0.5,
    'retargeting': 0.3,
    'organic': 0.4,
    'email': 0.4,
    'media_ad': 0.5,
    'marketplace': 0.5
}

In [19]:
def get_attribution_data_pos_sample(i, N, traffic_sources_coeffs, where_condition = ''):
    q = '''
    SELECT 
        sum(VisitRevenue*SourceCoefNorm) as Revenue,
        sum(VisitPurchases*SourceCoefNorm) as Purchases,
        sum(SourceCoefNorm) as Visits,
        Source,
        SourceDetailed
    FROM
        (SELECT 
            groupArray(Source) as Sources,
            groupArray(SourceDetailed) as SourcesDetailed,
            VisitID,
            any(ClientID) as ClientID,
            any(DateTime) as StartTime,
            any(VisitRevenue) as VisitRevenue,
            any(VisitPurchases) as VisitPurchases,
            arrayMap(x -> transform(x, [{traffic_sources}], [{traffic_sources_coeffs}], 0), Sources) as SourcesRawCoefs,
            arrayMap(x -> 1/(length(SourcesRawCoefs) - x), range(length(SourcesRawCoefs))) as PositionCoefs,
            arrayMap(x, y -> x * y, SourcesRawCoefs, PositionCoefs) as SourcesCoefs,
            arraySum(SourcesCoefs) as SourcesCoefsSum,
            arrayMap(x -> x/SourcesCoefsSum, SourcesCoefs) as SourcesCoefsNorm
        FROM
            (SELECT
                ClientID,
                DateTime,
                VisitRevenue,
                VisitPurchases,
                SourceTime,
                Source,
                SourceDetailed,
                VisitID,
                LastSearchEngine,
                LastAdvEngine
            FROM
                (SELECT
                    ClientID,
                    DateTime,
                    arraySum(PurchaseRevenue) as VisitRevenue,
                    length(PurchaseID) as VisitPurchases,
                    LastTraficSource,
                    VisitID,
                    LastSearchEngine,
                    LastAdvEngine
                FROM visits_all SAMPLE 1/{N} OFFSET {i}/{N}
                WHERE ClientID != 0)
            ALL LEFT JOIN
                (SELECT
                    ClientID,
                    DateTime as SourceTime,
                    if(
                        lower(UTMMedium) IN ('cpa', 'cpc', 'marketplace'), 
                        lower(UTMMedium),
                        if(
                            LastTraficSource = 'ad', 
                            if(
                                LastAdvEngine = 'market', 
                                'marketplace',
                                if(
                                    (DirectConditionType = 2) OR (extractURLParameter(StartURL, 'ref') = 'ga_ref'),
                                    'retargeting',
                                    'media_ad')
                            ),
                            if(
                               LastTraficSource IN ('direct', 'saved', 'undefined', 'internal'),
                               'direct',
                               LastTraficSource
                            ) 
                        )
                    ) as Source,
                    if(Source = 'organic', 
                        if(LastSearchEngine IN ('yandex', 'google'), concat('organic ', LastSearchEngine), 'organic other'),
                        if(Source IN ('media_ad', 'retargeting', 'cpa', 'cpc'),
                            if(
                                LastAdvEngine IN ('market', 'google_adwords', 'yandex'), 
                                concat(Source, concat(' ', LastAdvEngine)),
                                concat(Source, ' other')
                            ),
                            Source
                        )) as SourceDetailed
                 FROM visits_all SAMPLE 1/{N} OFFSET {i}/{N} {where_condition})
            USING ClientID
            WHERE SourceTime <= DateTime
            ORDER BY SourceTime)
        GROUP BY VisitID)
    ARRAY JOIN
        SourcesCoefsNorm as SourceCoefNorm,
        Sources as Source,
        SourcesDetailed as SourceDetailed
    GROUP BY Source, SourceDetailed
    ORDER BY Visits DESC
    FORMAT TabSeparatedWithNames
    '''.format(
        traffic_sources = ', '.join(map(lambda x: "'%s'" % x, traffic_sources_coeffs.keys())),
        traffic_sources_coeffs = ', '.join(map(str, traffic_sources_coeffs.values())),
        i = i, N = N,
        where_condition = where_condition
    )
    # print q
    return get_clickhouse_df(q)

def get_attribution_data_pos(traffic_sources_coeffs, where_condition=''):
    tmp_dfs = []
    for i in range(10):
        tmp_dfs.append(get_attribution_data_pos_sample(i, 10, traffic_sources_coeffs, where_condition))
    
    return pd.concat(tmp_dfs).groupby(['Source', 'SourceDetailed']).sum()

In [20]:
%time not_bounce_pos_df = get_attribution_data_pos(traffic_sources_coeffs, where_condition = 'WHERE IsBounce = 0')

CPU times: user 80.3 ms, sys: 10.3 ms, total: 90.6 ms
Wall time: 33.1 s


In [21]:
not_bounce_pos_df_norm = not_bounce_pos_df.apply(lambda x: 100.*x/not_bounce_pos_df.sum(), axis = 1)

## Comparison

For visualization we'll use `plot.ly` library that allows to get interactive graphs using just python.

In [22]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go
import requests
import StringIO
import pandas as pd

print __version__ # need 1.9.0 or greater

init_notebook_mode(connected = True)

def show_bar_plot(cmp_df):
    data = []
    for column in cmp_df.columns:
        trace = go.Bar(
            x = cmp_df.index.values,
            y = cmp_df[column].values,
            name = column
        )
        data.append(trace)
    layout = go.Layout({'xaxis': {'tickangle': 45}})
    fig = go.Figure(data = data, layout = layout)
    iplot(fig, show_link=False)

1.12.9


In [23]:
dct = {
    'last click': last_raw_df_norm,
    'behaviour': not_bounce_df_norm,
    'time decay & weights': not_bounce_pos_df_norm,
}

In [24]:
def get_comparison(parameter, detalization, show_delta = False):
    cmp_df = pd.DataFrame()
    for item in dct:
        cmp_df[item] = dct[item].reset_index().groupby(detalization)[parameter].sum()
    cmp_df = cmp_df.sort_values('last click', ascending = False)
    if not show_delta:
        return cmp_df
    cmp_df_norm = cmp_df.apply(lambda x: x - cmp_df['last click']).drop('last click', axis = 1).sort_values('behaviour')
    return cmp_df_norm

In [25]:
show_bar_plot(get_comparison('Revenue', 'Source', show_delta=False))

In [26]:
show_bar_plot(get_comparison('Revenue', 'Source', show_delta=True))

In [27]:
show_bar_plot(get_comparison('Revenue', 'SourceDetailed'))

In [28]:
show_bar_plot(get_comparison('Revenue', 'SourceDetailed', show_delta=True))